In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np

from src.utils.logger import Logging
from src.nn.pde import helmholtz_operator
from src.utils.plot_prediction import plt_prediction
from src.data.helmholtz_dataset import u as u_hlemholtz
from src.data.helmholtz_dataset import f as f_helmholtz
import src.trainer.helmholtz_train as helmholtz_train
from src.nn.DVPDESolver import DVPDESolver
from src.nn.CVPDESolver import CVPDESolver
from src.nn.ClassicalSolver import ClassicalSolver

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [10]:
log_path = "./checkpoints/helmholtz"
logger = Logging(log_path)

In [13]:

mode = "hybrid"
num_qubits = 5
output_dim = 1
input_dim = 2
hidden_dim = 50
num_quantum_layers = 1
cutoff_dim = 10
classic_network = [input_dim, hidden_dim, output_dim]


args = {
    "batch_size": 64,
    "epochs": 20000,
    "lr": 0.0001,
    "seeds": [12345 , 42, 123, 456, 789, 1011 , 1 ,1234 ],
    "print_every": 10000,
    "log_path": log_path,
    "input_dim": input_dim,
    "output_dim": output_dim,
    "num_qubits": num_qubits,
    "hidden_dim": hidden_dim,
    "num_quantum_layers": num_quantum_layers,
    "classic_network": classic_network,
    "q_ansatz": "layered_circuit",  # options: "alternating_layer_tdcnot", "abbas" , farhi , sim_circ_13_half, sim_circ_13 , sim_circ_14_half, sim_circ_14 , sim_circ_15 ,sim_circ_19
    "mode": mode,
    "activation": "tanh",  # options: "null", "partial_measurement_half" , partial_measurement_x
    "shots": None,  # Analytical gradients enabled
    "problem": "helmholtz",
    "solver": "DV",
    "device": DEVICE,
    "method": "None",
    "cutoff_dim": cutoff_dim,  # num_qubits >= cutoff_dim
    "class": "DVQuantumLayer",  # options : "DVQuantumLayer", "CVQuantumLayer", "Enhanced_CVQuantumLayer"
    "encoding": "None",
}


In [ ]:
A1 = 1
A2 = 4
LAMBDA = 1.0
num_points = 100

dom_coords = torch.tensor([[-1.0, -1.0], [1.0, 1.0]], dtype=torch.float32).to(DEVICE)

t = (
    torch.linspace(dom_coords[0, 0], dom_coords[1, 0], num_points)
    .to(DEVICE)
    .unsqueeze(1)
)

x = (
    torch.linspace(dom_coords[0, 1], dom_coords[1, 1], num_points)
    .to(DEVICE)
    .unsqueeze(1)
)
t, x = torch.meshgrid(t.squeeze(), x.squeeze())
X_star = torch.hstack((t.flatten().unsqueeze(1), x.flatten().unsqueeze(1))).to(DEVICE)
u_star = u_hlemholtz(X_star, A1, A2)
f_star = f_helmholtz(X_star, A1, A2, LAMBDA)


results = {}
for seed in args["seeds"]:
    torch.manual_seed(seed)
    np.random.seed(seed)
        
    if args["solver"] == "CV":
        model = CVPDESolver(args, logger, X_star, DEVICE)
        model.logger.print("Using CV Solver")
    if args["solver"] == "DV":
        model = DVPDESolver(args, logger, X_star, DEVICE)
        model.logger.print("Using DV Solver")
    else :
        model = ClassicalSolver(args, logger, X_star, DEVICE)
        model.logger.print("Using Classical Solver")
            
    helmholtz_train.train(model)
    model.save_state()
    model.logger.print("Training completed successfuly!")
    u_pred_star, f_pred_star = helmholtz_operator(model, X_star[:, 0:1], X_star[:, 1:2])

    error_u = torch.norm(u_pred_star - u_star, 2) / torch.norm(u_star, 2) * 100
    error_f = torch.norm(f_pred_star - f_star, 2) / torch.norm(f_star, 2) * 100
    logger.print("Seed: {}".format(seed))
    logger.print("Relative L2 error_u: {:.2e}".format(error_u.item()))
    logger.print("Relative L2 error_f: {:.2e}".format(error_f.item()))
    
    results[seed] = (error_u, error_f)
    del model

INFO:src.utils.logger:Using DV Solver
INFO:src.utils.logger:The circuit used in the study:


The circuit is saved in ./checkpoints/helmholtz/2025-09-21_16-12-26-839891


INFO:src.utils.logger:Iteration: 0, loss_r = 7.6e+03 ,  loss_bc = 9.1e-03,  lr = 1.0e-04, time_taken = 1.4e+01
INFO:src.utils.logger:Model state saved to ./checkpoints/helmholtz/2025-09-21_16-12-26-839891/model.pth


In [5]:

state = {
    "args": args,
    "results": results
}

model_path = f"models/stability_results_helmholtz_{args['solver']}.pth"

with open(model_path, "wb") as f:
    torch.save(state, f)

logger.print(f"Model state saved to {model_path}")


INFO:src.utils.logger:Model state saved to models/stability_results_helmholtz_DV.pth


In [6]:



plt_prediction(
    logger,
    X_star.cpu().detach().numpy(),
    u_star.cpu().detach().numpy(),
    u_pred_star.cpu().detach().numpy(),
    f_star.cpu().detach().numpy(),
    f_pred_star.cpu().detach().numpy(),
)



NameError: name 'u_pred_star' is not defined